In [1]:
%matplotlib inline
import numpy as np
import scipy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

sns.set(color_codes=True)
sns.set(rc={"figure.figsize": (16, 8)}); 
np.random.seed(20170619)

In [2]:
wards = pd.read_excel("./Coll.centre key_v3.xlsx")
ICU_wards = wards[wards['ICU']==1.0]
General_wards = wards[wards['General Ward']==1.0]
exclude = wards[wards['To exclude']==1.0]
location = 'Coll.Centre'
targets = list(ICU_wards[location])
list(ICU_wards[location])

['NICU', 'PAED HDU', 'PICU']

In [24]:
data = pd.read_pickle("./data11.pkl")

In [30]:
# 'Lab test name': [[current value min, max], [proposed value min, max]]
valid_labs = {
    'SODIUM': [[130.0,150.0], [130.82, 148.21]],
    'BICARBONATE': [[15.0, 35.0], [12.74, 36.45]],
    'CHLORIDE': [[90.0, 115.0], [91.35, 114.91]],
    'GLUCOSE': [[2.5, 9.0], [False, 17.04]], 
    'LACTATE': [[False, 10.0], [False, 5.06]], 
    'MAGNESIUM': [[0.6, 1.3], [0.58, 1.2]], 
    'PHOSPHATE': [[0.5, 3.0], [0.39, 2.6]], 
    'POTASSIUM': [[3.0, 6.0], [2.37, 6.33]], 
    #CORRECTED CALCIUM | [2.03, 3.01]      | [2.08, 3.03]
    'UREA' : [[False, 12.0], [False, 11.63]], 
    'SERUM CREAT': [[False, 120.0], [False, 129.18]], 
    'CALCIUM TOTAL': [[1.8, 3.0], [1.93, 3.25]]
}

In [32]:
data['included']

In [33]:
for lab, tworange in valid_labs.items():
    _min = lab + "_min"
    _max = lab + "_max"
    
    #if lab == "SODIUM":
    curr_min = valid_labs[lab][0][0] if valid_labs[lab][0][0] else False
    curr_max = valid_labs[lab][0][1] if valid_labs[lab][0][1] else False
    prop_min = valid_labs[lab][1][0] if valid_labs[lab][1][0] else False
    prop_max = valid_labs[lab][1][1] if valid_labs[lab][1][1] else False
        
    select_curr_max = [ i and j for i, j in zip(data['included'], data[_max] >= curr_max)] if curr_max else [False for i in data['included']]
    select_curr_min =  [ i and j for i, j in zip(data['included'], data[_min] <= curr_min)] if curr_min else [False for i in data['included']]
    select_prop_max = [ i and j for i, j in zip(data['included'], data[_max] >= prop_max)] if curr_max else [False for i in data['included']]
    select_prop_min = [ i and j for i, j in zip(data['included'], data[_min] <= prop_min)] if curr_max else [False for i in data['included']]
        
    total = float(data.loc[data['included'], ['Research.ID', 'age', location, 'Collected', lab, lab+"_max", lab+"_min", 'included']].dropna()['Research.ID'].unique().shape[0])
    flaged_curr_min = float(data.loc[select_curr_min, 'Research.ID'].unique().shape[0])
    flaged_curr_max = float(data.loc[select_curr_max, 'Research.ID'].unique().shape[0])
    flaged_prop_min = float(data.loc[select_prop_min, 'Research.ID'].unique().shape[0])
    flaged_prop_max = float(data.loc[select_prop_max, 'Research.ID'].unique().shape[0])
        
    print("\n"+lab)
    print("Total patients transfered to ICU: {0}".format(total))
    print("Flaged by current min: {0}, {1}%".format(flaged_curr_min, np.round(flaged_curr_min / total * 100.0, decimals=2)))
    print("Flaged by proposed min: {0}, {1}%".format(flaged_prop_min, np.round(flaged_prop_min / total * 100.0, decimals=2)))
    print("Difference proposed - current: {0}".format(np.round(flaged_prop_min / total * 100.0, decimals=2)-np.round(flaged_curr_min / total * 100.0, decimals=2)))
    print("Flaged by current max: {0}, {1}%".format(flaged_curr_max, np.round(flaged_curr_max / total * 100.0, decimals=2)))
    print("Flaged by proposed max: {0}, {1}%".format(flaged_prop_max, np.round(flaged_prop_max / total * 100.0, decimals=2)))
    print("Difference proposed - current: {0}".format(np.round(flaged_prop_max / total * 100.0, decimals=2)-np.round(flaged_curr_max / total * 100.0, decimals=2)))


LACTATE
Total patients transfered to ICU: 16.0
Flaged by current min: 0.0, 0.0%
Flaged by proposed min: 0.0, 0.0%
Difference proposed - current: 0.0
Flaged by current max: 0.0, 0.0%
Flaged by proposed max: 2.0, 12.5%
Difference proposed - current: 12.5

POTASSIUM
Total patients transfered to ICU: 640.0
Flaged by current min: 23.0, 3.59%
Flaged by proposed min: 2.0, 0.31%
Difference proposed - current: -3.28
Flaged by current max: 12.0, 1.88%
Flaged by proposed max: 7.0, 1.09%
Difference proposed - current: -0.7899999999999998

SODIUM
Total patients transfered to ICU: 640.0
Flaged by current min: 2.0, 0.31%
Flaged by proposed min: 2.0, 0.31%
Difference proposed - current: 0.0
Flaged by current max: 8.0, 1.25%
Flaged by proposed max: 12.0, 1.88%
Difference proposed - current: 0.6299999999999999

CHLORIDE
Total patients transfered to ICU: 640.0
Flaged by current min: 6.0, 0.94%
Flaged by proposed min: 10.0, 1.56%
Difference proposed - current: 0.6200000000000001
Flaged by current max: 33

In [34]:
for lab, tworange in valid_labs.items():
    _min = lab + "_min"
    _max = lab + "_max"
    
    #if lab == "SODIUM":
    curr_min = valid_labs[lab][0][0] if valid_labs[lab][0][0] else False
    curr_max = valid_labs[lab][0][1] if valid_labs[lab][0][1] else False
    prop_min = valid_labs[lab][1][0] if valid_labs[lab][1][0] else False
    prop_max = valid_labs[lab][1][1] if valid_labs[lab][1][1] else False
        
    select_curr_max = [ i  for i in data[_max] >= curr_max] if curr_max else [False for i in data['included']]
    select_curr_min =  [ i  for i in data[_min] <= curr_min] if curr_min else [False for i in data['included']]
    select_prop_max = [ i  for i in data[_max] >= prop_max] if curr_max else [False for i in data['included']]
    select_prop_min = [ i for i in data[_min] <= prop_min] if curr_max else [False for i in data['included']]
        
    total = float(data.loc[:, ['Research.ID', 'age', location, 'Collected', lab, 'included']].dropna()['Research.ID'].unique().shape[0])
    flaged_curr_min = float(data.loc[select_curr_min, 'Research.ID'].unique().shape[0])
    flaged_curr_max = float(data.loc[select_curr_max, 'Research.ID'].unique().shape[0])
    flaged_prop_min = float(data.loc[select_prop_min, 'Research.ID'].unique().shape[0])
    flaged_prop_max = float(data.loc[select_prop_max, 'Research.ID'].unique().shape[0])
        
    print("\n"+lab)
    print("Total patients: {0}".format(total))
    print("Flaged by current min: {0}, {1}%".format(flaged_curr_min, np.round(flaged_curr_min / total * 100.0, decimals=2)))
    print("Flaged by proposed min: {0}, {1}%".format(flaged_prop_min, np.round(flaged_prop_min / total * 100.0, decimals=2)))
    print("Difference proposed - current: {0}".format(np.round(flaged_prop_min / total * 100.0, decimals=2)-np.round(flaged_curr_min / total * 100.0, decimals=2)))
    print("Flaged by current max: {0}, {1}%".format(flaged_curr_max, np.round(flaged_curr_max / total * 100.0, decimals=2)))
    print("Flaged by proposed max: {0}, {1}%".format(flaged_prop_max, np.round(flaged_prop_max / total * 100.0, decimals=2)))
    print("Difference proposed - current: {0}".format(np.round(flaged_prop_max / total * 100.0, decimals=2)-np.round(flaged_curr_max / total * 100.0, decimals=2)))



LACTATE
Total patients: 1691.0
Flaged by current min: 0.0, 0.0%
Flaged by proposed min: 0.0, 0.0%
Difference proposed - current: 0.0
Flaged by current max: 2.0, 0.12%
Flaged by proposed max: 22.0, 1.3%
Difference proposed - current: 1.1800000000000002

POTASSIUM
Total patients: 17755.0
Flaged by current min: 160.0, 0.9%
Flaged by proposed min: 13.0, 0.07%
Difference proposed - current: -0.8300000000000001
Flaged by current max: 155.0, 0.87%
Flaged by proposed max: 64.0, 0.36%
Difference proposed - current: -0.51

SODIUM
Total patients: 17772.0
Flaged by current min: 75.0, 0.42%
Flaged by proposed min: 75.0, 0.42%
Difference proposed - current: 0.0
Flaged by current max: 40.0, 0.23%
Flaged by proposed max: 56.0, 0.32%
Difference proposed - current: 0.09

CHLORIDE
Total patients: 17772.0
Flaged by current min: 32.0, 0.18%
Flaged by proposed min: 42.0, 0.24%
Difference proposed - current: 0.06
Flaged by current max: 105.0, 0.59%
Flaged by proposed max: 105.0, 0.59%
Difference proposed - 